In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [2]:
def make_features(df, max_lag, rolling_mean):
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['dayofweek'] = df.index.dayofweek
    for lag in range(1,max_lag+1):
        df[f'lag_{lag}'] = df['PJME_MW'].shift(lag)
    df['rolling_mean'] = df['PJME_MW'].shift(1).rolling(rolling_mean).mean()

In [25]:
data = pd.read_csv('/home/stepbruh/Рабочий стол/JetBrains Projects/DataspellProjects/Projects/Data Scientist Plus/Part_14/Анализ временных рядов/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

make_features(data,6,1)
print(data.head())

             PJME_MW  year  month  day  dayofweek     lag_1     lag_2  \
Datetime                                                                
2002-01-01  714857.0  2002      1    1          1       NaN       NaN   
2002-01-02  822277.0  2002      1    2          2  714857.0       NaN   
2002-01-03  828285.0  2002      1    3          3  822277.0  714857.0   
2002-01-04  809171.0  2002      1    4          4  828285.0  822277.0   
2002-01-05  729723.0  2002      1    5          5  809171.0  828285.0   

               lag_3     lag_4  lag_5  lag_6  rolling_mean  
Datetime                                                    
2002-01-01       NaN       NaN    NaN    NaN           NaN  
2002-01-02       NaN       NaN    NaN    NaN      714857.0  
2002-01-03       NaN       NaN    NaN    NaN      822277.0  
2002-01-04  714857.0       NaN    NaN    NaN      828285.0  
2002-01-05  822277.0  714857.0    NaN    NaN      809171.0  


In [26]:
data = data.dropna()

In [27]:
train, test = train_test_split(data, shuffle=False, test_size=.2)

In [28]:
print(train.shape)
print(test.shape)

(4842, 12)
(1211, 12)


In [29]:
train_features = train.drop('PJME_MW', axis=1)
train_target = train['PJME_MW']
test_features = test.drop('PJME_MW', axis=1)
test_target = test['PJME_MW']

In [30]:
model = LinearRegression()
model.fit(train_features, train_target)
pred = model.predict(test_features)
mae = mean_absolute_error(test_target, pred)
mae

36822.485686357584